# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
import functions_notebook_arinew as fan
from datetime import timedelta
from scipy.stats import entropy


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

In [3]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_1)

In [4]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_15m_05h = fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)

In [5]:
df_cc = pd.DataFrame(features_cc)
df = pd.DataFrame(features_pf_15m_05h)

In [6]:
features_cc.clear()
features_pf_15m_05h.clear()

In [7]:
fn.calcolo_features_binarie(df_cc)
fn.calcolo_features_binarie(df)

In [8]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'])
df['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])

In [9]:
df_cc.sort_values(by='frame.frame.time_utc')
df.sort_values(by='frame.frame.time_utc')

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,icmpv6,ipv6,eth,ip,arp,mbtcp,stp,icmp,data,tcp
0,1,"May 21, 2018 13:12:37.348723000 ora legale Eur...",2018-05-21 11:12:37.348723+00:00,1526901157.348723000,0.000000000,0.000000000,0.000000000,0.000000000,1,60,...,0,0,1,0,1,0,0,0,0,0
1,1,"May 21, 2018 13:12:37.350527000 ora legale Eur...",2018-05-21 11:12:37.350527+00:00,1526901157.350527000,0.000000000,0.001804000,0.001804000,0.001804000,2,64,...,0,0,1,0,1,0,0,0,0,0
2,1,"May 21, 2018 13:12:37.350544000 ora legale Eur...",2018-05-21 11:12:37.350544+00:00,1526901157.350544000,0.000000000,0.000017000,0.000017000,0.001821000,3,60,...,0,0,1,1,0,0,0,0,0,1
3,1,"May 21, 2018 13:12:37.360801000 ora legale Eur...",2018-05-21 11:12:37.360801+00:00,1526901157.360801000,0.000000000,0.010257000,0.010257000,0.012078000,4,60,...,0,0,1,1,0,0,0,0,0,1
4,1,"May 21, 2018 13:12:37.360805000 ora legale Eur...",2018-05-21 11:12:37.360805+00:00,1526901157.360805000,0.000000000,0.000004000,0.000004000,0.012082000,5,60,...,0,0,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197773,1,"May 21, 2018 13:42:36.345657000 ora legale Eur...",2018-05-21 11:42:36.345657+00:00,1526902956.345657000,0.000000000,0.001220000,0.001220000,1798.996934000,197774,60,...,0,0,1,1,0,0,0,0,0,1
197774,1,"May 21, 2018 13:42:36.345673000 ora legale Eur...",2018-05-21 11:42:36.345673+00:00,1526902956.345673000,0.000000000,0.000016000,0.000016000,1798.996950000,197775,60,...,0,0,1,1,0,0,0,0,0,1
197775,1,"May 21, 2018 13:42:36.345854000 ora legale Eur...",2018-05-21 11:42:36.345854+00:00,1526902956.345854000,0.000000000,0.000181000,0.000181000,1798.997131000,197776,60,...,0,0,1,1,0,0,0,0,0,1
197776,1,"May 21, 2018 13:42:36.354401000 ora legale Eur...",2018-05-21 11:42:36.354401+00:00,1526902956.354401000,0.000000000,0.008547000,0.008547000,1799.005678000,197777,60,...,0,0,1,1,0,0,0,0,0,1


In [10]:
start_time = df['frame.frame.time_utc'].iloc[0]
end_time=start_time+timedelta(seconds=298.8)
end_time2 = end_time + timedelta(seconds=900)

In [11]:
df_cc1 = df[(df['frame.frame.time_utc'] >= start_time) & (df['frame.frame.time_utc'] < end_time)].copy()
df_attack = df[(df['frame.frame.time_utc'] >= end_time) & (df['frame.frame.time_utc'] <= end_time2)].copy()
df_cc2 = df[(df['frame.frame.time_utc'] > end_time2)].copy()

In [12]:
#df_cc_aggregation = fan.calcola_features(df_cc)
#df_cc1_aggregation = fan.calcola_features(df_cc1)
#df_cc2_aggregation = fan.calcola_features(df_cc2)
df_att_aggregation = fan.calcola_features(df_attack)

In [13]:
df_att_aggregation.filter(like='icmp')

,icmp_count,per_icmp_count,icmp_request_count,icmp_response_count,icmp_req_resp_fraction
0,184,0.847926,184,0.0,NaN
1,680,0.917679,680,0.0,NaN
2,994,0.965986,994,0.0,NaN
3,998,0.945972,998,0.0,NaN
4,996,0.952199,996,0.0,NaN
...,...,...,...,...,...
176,1000,0.965251,1000,0.0,NaN
177,1000,0.946074,1000,0.0,NaN
178,1000,0.952381,1000,0.0,NaN
179,990,0.964912,990,0.0,NaN


In [18]:
d = {'col1': [1, 2], 'col2': [3, 4]}
pippo = pd.DataFrame(data=d)
pippo['pippo'] = df_att_aggregation[(df_att_aggregation['icmp_response_count'] != 0)]['icmp_request_count']/df_att_aggregation[df_att_aggregation['icmp_response_count'] !=0]['icmp_response_count']

In [19]:
pippo

,col1,col2,pippo
0,1,3,NaN
1,2,4,NaN
